<a href="https://colab.research.google.com/github/OmarSyedK/AI-Resume-Optimization/blob/main/resumeopt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q google-generativeai

import google.generativeai as genai
import json
from google.colab import userdata

#Fetch the API Key
api_key = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=api_key)

#In case the API key is not found
if not api_key:
  print("ERROR: GOOGLE_API_KEY not found")


In [15]:
#Prompt


#Markdown Resume
md_resume = input("Markdown Resume: ")


#Job Description
job_description = input("Job Description: ")

prompt = f"""
I am providing you a markdown resume and a job description \
I want you to optimize the resume according to the job role and job description \
Make relevent changes to the resume in sections such as skills, projects, achievments, etc \
while keeping the unique qualifications and strengths. \
Return a JSON object containing two keys:
1. The output should state in depth about the changes you made to the resume and why you made them. \
2. Then return the optimized resume in markdown format.

### Here is the resume in Markdown:
{md_resume}

### Here is the job description:
{job_description}

Please modify the resume to:
- Contain keywords and phrases relevent to the job description
- Make sure the experiences are presented in a way that match the job description requirements.
- Maintain clarity, conciseness, and professionalism throughout.
- There should be no --implied-- or --likely--, everything should be a statement
- - The Markdown format MUST follow semantic structure so it renders cleanly in PDF:
  * `#` for the name
  * `##` for main sections (Summary, Education, Skills, Experience, Projects, Achievements, Certifications, Languages, etc.)
  * `###` for sub-sections (e.g., each job or project title)
  * Use bullet points (`-`) for responsibilities, skills, and achievements,etc
  * Avoid inline formatting hacks (like mixing emojis or excessive bolding for section headers)
  * Ensure consistent spacing and formatting throughout

"""



Markdown Resume: John Doe Software Engineer | San Francisco, CA | john.doe@email.com | (123) 456-7890 | github.com/johndoe | linkedin.com/in/johndoe Technical Skills Languages: Python, JavaScript (ES6+), TypeScript, Java, C++ Frameworks & Libraries: React, Node.js, Express, Django, Spring Boot Databases: PostgreSQL, MongoDB, MySQL, Redis DevOps & Tools: Docker, Kubernetes, AWS, Git, CI/CD (GitHub Actions, Jenkins) Other: REST APIs, GraphQL, Agile/Scrum, Test-Driven Development (TDD) Professional Experience Software Engineer — TechCorp Inc. San Francisco, CA | Jan 2021 – Present - Designed and implemented a scalable microservices architecture, enhancing system performance and reducing deployment times by 30%. - Developed and maintained robust RESTful APIs serving over 500,000 monthly active users, ensuring high availability and low latency. - Led a team of 4 engineers in migrating a monolithic application to a modern stack leveraging Node.js and React, resulting in a 40% performance inc

In [16]:
# ---Model and generation configuration---
system_instruction = "You are a helpful assistant and an expert in career coaching specializing in tailoring resumes."

#Setting up the model
model = genai.GenerativeModel(
  model_name = 'gemini-2.5-flash-lite',
  system_instruction = system_instruction
)

# Configuration for the generation call
generation_config = genai.GenerationConfig(
    temperature=0.25,
    response_mime_type = "application/json",
    response_schema = {
        "type":"OBJECT",
        "properties":{
            "explanation":{"type":"STRING"},
            "resume":{"type":"STRING"}
        },
        "required":["explanation","resume"]
    }
)

In [17]:
# ---Make API Call---
try:
  print(f"Optimizing your resume for {job_description}...")
  response = model.generate_content(
  prompt,
  generation_config = generation_config
  )
  output_data = json.loads(response.text)
# --- Extract Response---
  print("Changes Explained")
  print(output_data["explanation"])
  print("Optimized Resume")
  print(output_data["resume"])
except Exception as e:
  print(f"Error: {e}")

Optimizing your resume for Devops developer...
Changes Explained
The resume was optimized for a DevOps Developer role by emphasizing and rephrasing existing experience to align with DevOps principles and keywords. Key changes include: 

1.  **Summary/Introduction:** While not explicitly present in the original, if a summary were added, it would highlight expertise in CI/CD, cloud infrastructure, automation, and system reliability, directly addressing DevOps responsibilities. 
2.  **Technical Skills:** The 'DevOps & Tools' section was brought to the forefront and expanded. Keywords like 'CI/CD', 'Docker', 'Kubernetes', and 'AWS' were retained and their relevance to automation and infrastructure management was implicitly strengthened. 
3.  **Professional Experience (TechCorp Inc.):** 
    *   The first bullet point, 'Designed and implemented a scalable microservices architecture, enhancing system performance and reducing deployment times by 30%', was rephrased to highlight the *automatio

In [18]:
!!pip install markdown2 weasyprint

from weasyprint import HTML, CSS
import markdown2
from google.colab import files
import os

# --- 1. Paste your resume markdown here ---
updated_resume = output_data["resume"]



# --- 2. Define Professional CSS ---
resume_style = """
body {
    font-family: 'Segoe UI', 'Helvetica Neue', Helvetica, Arial, sans-serif;
    line-height: 1.6;
    color: #2c3e50;
    background-color: #fff;
    margin: 40px;
    font-size: 11pt;
}

h1 {
    font-size: 24pt;
    color: #1a1a1a;
    text-align: center;
    margin-bottom: 5px;
    font-weight: 400;
    letter-spacing: 0.5px;
}

/* Contact info under name */
.contact-info {
    text-align: center;
    font-size: 10pt;
    color: #555;
    margin-bottom: 20px;
}

h2 {
    font-size: 13pt;
    color: #ffffff;
    background-color: #2c3e50;
    padding: 6px 10px;
    margin-top: 28px;
    margin-bottom: 12px;
    border-radius: 4px;
    font-weight: 500;
}

ul {
    padding-left: 18px;
    margin-top: 5px;
    margin-bottom: 10px;
    list-style-type: disc;
}

li {
    margin-bottom: 6px;
}

p {
    margin: 0 0 6px 0;
}

b {
    color: #2c3e50;
}
"""


# --- 3. Convert Markdown → HTML ---
html_content = markdown2.markdown(updated_resume)

# --- 4. Generate PDF ---
output_pdf_path = 'new_resume.pdf' # Moved this line up
output_pdf = output_pdf_path
HTML(string=html_content).write_pdf(output_pdf, stylesheets=[CSS(string=resume_style)])

# --- 5. Download PDF ---
files.download(output_pdf)

DEBUG:fontTools.ttLib.ttFont:Reading 'maxp' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'maxp' table
DEBUG:fontTools.subset.timer:Took 0.003s to load 'maxp'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'maxp'
INFO:fontTools.subset:maxp pruned
DEBUG:fontTools.ttLib.ttFont:Reading 'cmap' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'cmap' table
DEBUG:fontTools.ttLib.ttFont:Reading 'post' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'post' table
DEBUG:fontTools.subset.timer:Took 0.005s to load 'cmap'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'cmap'
INFO:fontTools.subset:cmap pruned
INFO:fontTools.subset:fpgm dropped
INFO:fontTools.subset:prep dropped
INFO:fontTools.subset:cvt  dropped
INFO:fontTools.subset:kern dropped
DEBUG:fontTools.subset.timer:Took 0.000s to load 'post'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'post'
INFO:fontTools.subset:post pruned
INFO:fontTools.subset:GPOS dropped
INFO:fontTools.subset:GSUB dropped
DEBUG:f

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
from pypdf import PdfReader
from google.colab import files
import io

# Upload file
uploaded = files.upload()

# Get filename
filename = list(uploaded.keys())[0]

# Create PDF reader object
reader = PdfReader(io.BytesIO(uploaded[filename]))

# Print number of pages in PDF file
print(f"Total Pages: {len(reader.pages)}")

all_text = ""
for page in reader.pages:
  text = page.extract_text()
  if text not in all_text:
    all_text += text
print(all_text)




Saving new_resume (8).pdf to new_resume (8) (5).pdf
Total Pages: 2
John Doe
Software Engineer | San Francisco, CA | john.doe@email.com | (123) 456-7890 |
github.com/johndoe | linkedin.com/in/johndoe
Technical Skills
Languages: Python, JavaScript (ES6+), TypeScript, Java, C++
Frameworks & Libraries: React, Node.js, Express, Django, Spring Boot
Databases: PostgreSQL, MongoDB, MySQL, Redis
DevOps & Tools: Docker, Kubernetes, AWS, Git, CI/CD (GitHub Actions, Jenkins)
Other: REST APIs, GraphQL, Agile/Scrum, Test-Driven Development (TDD)
Professional Experience
Software Engineer — TechCorp Inc.
San Francisco, CA | Jan 2021 – Present - Designed and implemented a scalable
microservices architecture, enhancing system performance and reducing deployment
times by 30%. - Developed and maintained robust RESTful APIs serving over 500,000
monthly active users, ensuring high availability and low latency. - Led a team of 4
engineers in migrating a monolithic application to a modern stack leveraging Nod